In [20]:
import tensorflow as tf
import time

In [4]:
tf.__version__

'2.5.0'

In [7]:
class FileDataset(tf.data.Dataset):
    def read_file_batches(num_samples):
#         open file
        time.sleep(0.03)
        for sample_index in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)
            yield (sample_index,)
            
    def __new__(cls, num_samples = 3):
        return tf.data.Dataset.from_generator(
            cls.read_file_batches, output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )
    

In [23]:
object = FileDataset()

In [8]:
def benchmark(dataset, num_epochs = 2):
#     start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01) # GPU time slot
#     print("Execution time:", time.perf_counter() - start_time)

In [30]:
%%timeit
benchmark(FileDataset())

541 ms ± 65.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
# In this mode new thing is cpu can start running while gpu is training with one sample which is derived from Dataset class
%%timeit
benchmark(FileDataset().prefetch(1))

470 ms ± 67.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
# Majority of the time AUTOTUNE is used

%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

UsageError: Line magic function `%%timeit` not found.


In [11]:
dataset = tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())


0
1
2
3
4


In [12]:
dataset =dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [13]:
dataset = dataset.cache()
for d in dataset.as_numpy_iterator():
    print(d)


0
1
4
9
16


In [14]:
# OR using list
dataset = dataset.cache()
list(dataset.as_numpy_iterator()) # reading it from cache

[0, 1, 4, 9, 16]

In [15]:

# applying for FileDataset()
def mapped_function(s):
    tf.py_function(lambda:time.sleep(0.03), [], ())
    return s
    


In [79]:
# benchmark this function and run this for 5 epochs
%%timeit
benchmark(FileDataset().map(mapped_function), 5)

UsageError: Line magic function `%%timeit` not found.


In [2]:
# benchmark this function and run this for 5 epochs
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function), 5)

UsageError: Line magic function `%%timeit` not found.


In [16]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function), 5)

1.93 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [17]:
# Using cache it is taking less than half time

%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function).cache(), 5)

700 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
benchmark(
    tf.data.Dataset.range(2)
    .interleave(lambda _: FileDataset())
)